In [1]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

/home/chenli/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 247kB/s]  
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 25.6kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 454kB/s]
Downloading: 100%|██████████| 440M/440M [05:27<00:00, 1.35MB/s] 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model 

In [3]:
inputs

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [5]:
last_hidden_states.shape # 最后一层的维度情况

torch.Size([1, 8, 768])

In [25]:
outputs.pooler_output.shape # 最后一层的第一个CLS的维度情况

torch.Size([1, 768])

In [28]:
print(outputs.hidden_states)

None


In [29]:
outputs.attentions

In [22]:
x3 = x.squeeze(2)
print(x3.shape)

torch.Size([2, 1, 2, 1, 2, 1])


In [23]:
x4 = x.squeeze(3)
print(x4.shape)

torch.Size([2, 1, 2, 2, 1])


# 开始编写代码

In [37]:
import torch
import torch.utils.data as Data
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

#定义数据集,方便后续模型读取批量数据。
class Dataset(Data.Dataset):
    def __init__(self, data_path):
        self.data = self.load_data(data_path)
    
    # 核心要变的就是load_data这部分函数
    # 改造成适合自己任务的数据集
    def load_data(self, data_path):
        # 先加载本地数据集
        tmp_dataset = load_dataset('csv',data_files=data_path,split='train')
        Data = {}
        for idx, line in enumerate(tmp_dataset):
            sample = line
            Data[idx] = sample
        return Data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]    

In [38]:
test_dataset = Dataset('./test.csv')

Using custom data configuration default-7edeb8d80da73fdc


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1250.91it/s]
                            

Dataset csv downloaded and prepared to /home/chenli/.cache/huggingface/datasets/csv/default-7edeb8d80da73fdc/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


In [42]:
test_dataset.featurs

AttributeError: 'Dataset' object has no attribute 'featurs'

In [43]:
tmp_dataset = load_dataset('csv',data_files='./test.csv',split='train')

Using custom data configuration default-7edeb8d80da73fdc
Found cached dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-7edeb8d80da73fdc/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


In [44]:
tmp_dataset.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [45]:
tmp_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

# 情感分类模型
trainer的使用

## 加载分词工具

In [46]:
from transformers import AutoTokenizer

# 加载分词工具
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

## 定义数据集

In [49]:
from datasets import load_dataset
from datasets import load_from_disk

datasets = load_from_disk('../SentimentAnalysis/model/data/ChnSentiCorp')
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

In [50]:
# 分词
def tokenizer_function(data):
    return tokenizer(data['text'],padding='max_length',truncation=True,max_length=50)

In [51]:
datasets = datasets.map(tokenizer_function,batched=True,batch_size=4)
datasets

100%|██████████| 300/300 [00:00<00:00, 302.68ba/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
})

In [52]:
# 取数据子集，否则数据太多跑不动
dataset_train = datasets['train'].shuffle().select(range(1000))
dataset_test = datasets['validation'].shuffle().select(range(200))

In [53]:
dataset_train

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

## 加载模型

In [54]:
from transformers import AutoModelForSequenceClassification

# 加载模型
model = AutoModelForSequenceClassification.from_pretrained('bert-base-chinese',num_labels=2)
print(sum([i.nelement() for i in model.parameters()])/10000)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

10226.9186


In [61]:
total = sum(p.numel() for p in model.parameters())
print("total param:",total)

total param: 102269186


## 定义评价函数

In [55]:
import numpy as np
from datasets import load_metric
from transformers.trainer_utils import EvalPrediction

In [56]:
def compute_metrics(eval_preds):
    metric = load_metric('glue','mrpc')
    logits,labels = eval_preds # 预测值和真实值
    predictions = np.argmax(logits,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [60]:
# 模拟测试输出
eval_pred = EvalPrediction(
    predictions=np.array([[0,1],[2,3],[4,5],[6,7]]),
    label_ids=np.array([1,1,1,1])
)
compute_metrics(eval_pred)

ConnectionError: Couldn't reach https://raw.githubusercontent.com/huggingface/datasets/2.5.2/metrics/glue/glue.py (ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Read timed out. (read timeout=100)")))

In [58]:
pip install scipy sklearn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 27.4 kB/s eta 0:00:0000:0100:30
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 30.4 kB/s eta 0:00:0000:0100:37
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 39.3 kB/s eta 0:00:00a 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=32fef8345eccfb67476084b35919d18338f273b9410cac1d1723478768f3bbe0
  Stored in directory: /home/chenli/.cache/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


## 定义训练器并测试

In [63]:
from transformers import TrainingArguments,Trainer

In [64]:
# 初始化训练参数
args = TrainingArguments(output_dir='./output_dir',evaluation_strategy='epoch') # evaluation_strategy表示每隔多长时间进行一次测试
args.num_train_epoch = 1 # 训练的轮次
args.learning_rate = 1e-5 # 学习率
args.weight_decay = 1e-2
args.per_device_eval_batch_size = 32
args.per_device_train_batch_size = 16

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [66]:
help(args)

Help on TrainingArguments in module transformers.training_args object:

class TrainingArguments(builtins.object)
 |  TrainingArguments(output_dir: str, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, evaluation_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', warmup_ratio: float = 0.0, w

In [67]:
# 初始化训练器
trainer = Trainer(
    model = model,
    args = args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics
)

In [68]:
# 评价模型
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 32


{'eval_loss': 0.7413558959960938,
 'eval_accuracy': 0.54,
 'eval_f1': 0.0,
 'eval_runtime': 17.3016,
 'eval_samples_per_second': 11.56,
 'eval_steps_per_second': 0.405}

In [69]:
# 训练
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 189


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.362678,0.850000,0.835165
2,No log,0.368508,0.850000,0.831461
3,No log,0.355735,0.865000,0.855615


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 32


Training compl

TrainOutput(global_step=189, training_loss=0.3308770315987723, metrics={'train_runtime': 889.136, 'train_samples_per_second': 3.374, 'train_steps_per_second': 0.213, 'total_flos': 77083317000000.0, 'train_loss': 0.3308770315987723, 'epoch': 3.0})

In [70]:
# 评价模型
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 32


Using the latest cached version of the module from /home/chenli/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Mon Nov  7 19:47:24 2022) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.


{'eval_loss': 0.3557354807853699,
 'eval_accuracy': 0.865,
 'eval_f1': 0.8556149732620322,
 'eval_runtime': 117.9623,
 'eval_samples_per_second': 1.695,
 'eval_steps_per_second': 0.059,
 'epoch': 3.0}

In [71]:
# 保存模型参数
# 保存的是模型参数，而不是模型本身
trainer.save_model(output_dir='./output_dir')

Saving model checkpoint to ./output_dir
Configuration saved in ./output_dir/config.json
Model weights saved in ./output_dir/pytorch_model.bin


## 使用模型参数

In [73]:
import torch

def collate_fn(data):
    label = [i['label'] for i in data]
    input_ids = [i['input_ids'] for i in data]
    token_type_ids = [i['token_type_ids'] for i in data]
    attention_mask = [i['attention_mask'] for i in data]
    
    label = torch.LongTensor(label)
    input_ids = torch.LongTensor(input_ids)
    token_type_ids = torch.LongTensor(token_type_ids)
    attention_mask = torch.LongTensor(attention_mask)
    
    return label,input_ids,token_type_ids,attention_mask

In [74]:
# 数据加载
loader_test = torch.utils.data.DataLoader(dataset=dataset_test,batch_size=4,collate_fn=collate_fn,shuffle=True,drop_last=True)

In [76]:
for i,(label,input_ids,token_type_ids,attention_mask) in enumerate(loader_test):
    break

label,input_ids,token_type_ids,attention_mask

(tensor([1, 0, 0, 0]),
 tensor([[ 101, 1912, 6225, 4023,  778, 8024, 4635, 5682, 1469, 6844, 1394, 1957,
          4495, 4500,  511, 5314, 4638,  691,  691, 3683, 6772, 6639, 8024, 7674,
          1044, 4692,  677,  749,  100, 4638, 4801, 4669,  511, 3683, 2669, 3249,
          4638, 2487, 8024, 2798, 5314,  100, 4801, 4669, 1315,  886, 3221, 1743,
          2578,  102],
         [ 101, 2940,  741, 6206, 3724, 2145, 2787, 5632, 2346, 6934, 2164, 1168,
          1266,  776, 8024, 1045, 6934, 6589, 2218, 4685, 2496,  754,  741, 6589,
          8024, 6820, 3300, 2553, 6206, 2940, 1408, 8024, 3173,  743,  671, 3315,
           679, 3291, 1962, 1408, 8024,  872,  812, 6821, 3416, 4638, 3302, 1218,
          2523,  102],
         [ 101, 6963, 3221, 6443, 6432, 4638, 1962, 1557, 8024, 2769,  738,  743,
           749,  511, 1377, 3221, 4692,  679, 2743, 8024, 2769, 1968, 4692,  749,
           738, 6432, 3766, 1567, 4500,  511, 1511,  511,  511,  511,  511,  511,
           511,  511,  511,  

In [81]:
# 测试
import torch

def test():
    model.load_state_dict(torch.load('./output_dir/pytorch_model.bin')) # 加载模型
    
    model.eval() # 把模型切换到运行模式
    
    # 运算
    out = model(input_ids=input_ids,token_type_ids=token_type_ids,attention_mask=attention_mask)
    
    # logits是一个 [4,2] 的矩阵 -> [4],变成四个结果
    out = out['logits'].argmax(dim=1)
    
    correct = (out==label).sum().item()
    
    return correct/len(label)
                                      

In [82]:
test()

1.0